In [1]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [3]:
ipl = pd.read_csv(
    r"C:\Users\ADMIN\Desktop\ipl_score\data\cleaned\ipl_clean.csv",
    low_memory=False
)


In [4]:
#player-match aggregation

df = (ipl.groupby(['batter', 'match_id', 'date', 'venue', 'bowler','bowling_team'])
    .agg({'batsman_runs': 'sum','is_wicket': 'sum'})
    .reset_index())


In [5]:
df = df.sort_values(['batter','date'])

In [6]:
print(df)

               batter  match_id        date                            venue  \
0      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
1      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
2      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
3      A Ashish Reddy    548352  2012-05-04  MA Chidambaram Stadium, Chepauk   
4      A Ashish Reddy    548352  2012-05-04  MA Chidambaram Stadium, Chepauk   
...               ...       ...         ...                              ...   
54066          Z Khan   1082595  2017-04-08            M.Chinnaswamy Stadium   
54067          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54068          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54069          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54070          Z Khan   1082646  2017-05-14                 Feroz Shah Kotla   

                bowler                 

In [7]:
# rolling form feature

df['rolling_avg_5'] = ( df.groupby('batter')['batsman_runs'].rolling(5)
    .mean().reset_index(level=0, drop=True))


In [8]:
print(df)

               batter  match_id        date                            venue  \
0      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
1      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
2      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
3      A Ashish Reddy    548352  2012-05-04  MA Chidambaram Stadium, Chepauk   
4      A Ashish Reddy    548352  2012-05-04  MA Chidambaram Stadium, Chepauk   
...               ...       ...         ...                              ...   
54066          Z Khan   1082595  2017-04-08            M.Chinnaswamy Stadium   
54067          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54068          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54069          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54070          Z Khan   1082646  2017-05-14                 Feroz Shah Kotla   

                bowler                 

In [9]:
# venue average feature
df['venue_avg'] = df.groupby(['batter','venue'])['batsman_runs'].transform('mean')

In [10]:
print(df)

               batter  match_id        date                            venue  \
0      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
1      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
2      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
3      A Ashish Reddy    548352  2012-05-04  MA Chidambaram Stadium, Chepauk   
4      A Ashish Reddy    548352  2012-05-04  MA Chidambaram Stadium, Chepauk   
...               ...       ...         ...                              ...   
54066          Z Khan   1082595  2017-04-08            M.Chinnaswamy Stadium   
54067          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54068          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54069          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54070          Z Khan   1082646  2017-05-14                 Feroz Shah Kotla   

                bowler                 

In [11]:
#opponent pvt feature
df['pvt_avg'] = df.groupby(['batter','bowling_team'])['batsman_runs'].expanding().mean().shift(1)\
                            .reset_index(level=[0,1],drop=True)

In [12]:
print(df)

               batter  match_id        date                            venue  \
0      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
1      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
2      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
3      A Ashish Reddy    548352  2012-05-04  MA Chidambaram Stadium, Chepauk   
4      A Ashish Reddy    548352  2012-05-04  MA Chidambaram Stadium, Chepauk   
...               ...       ...         ...                              ...   
54066          Z Khan   1082595  2017-04-08            M.Chinnaswamy Stadium   
54067          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54068          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54069          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54070          Z Khan   1082646  2017-05-14                 Feroz Shah Kotla   

                bowler                 

In [13]:
#pvp feature
df['pvp_avg'] = (df.groupby(['batter', 'bowler'])['batsman_runs']
    .transform('mean'))


In [14]:
print(df)

               batter  match_id        date                            venue  \
0      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
1      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
2      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
3      A Ashish Reddy    548352  2012-05-04  MA Chidambaram Stadium, Chepauk   
4      A Ashish Reddy    548352  2012-05-04  MA Chidambaram Stadium, Chepauk   
...               ...       ...         ...                              ...   
54066          Z Khan   1082595  2017-04-08            M.Chinnaswamy Stadium   
54067          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54068          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54069          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54070          Z Khan   1082646  2017-05-14                 Feroz Shah Kotla   

                bowler                 

In [15]:
#career average
df['career_avg'] = df.groupby('batter')['batsman_runs'].expanding().mean().shift(1)\
                            .reset_index(level=0,drop=True)

In [16]:
print(df)

               batter  match_id        date                            venue  \
0      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
1      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
2      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
3      A Ashish Reddy    548352  2012-05-04  MA Chidambaram Stadium, Chepauk   
4      A Ashish Reddy    548352  2012-05-04  MA Chidambaram Stadium, Chepauk   
...               ...       ...         ...                              ...   
54066          Z Khan   1082595  2017-04-08            M.Chinnaswamy Stadium   
54067          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54068          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54069          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54070          Z Khan   1082646  2017-05-14                 Feroz Shah Kotla   

                bowler                 

In [17]:
# Fill missing values (NaN) in the 'rolling_avg_5' column with the corresponding 'career_avg' values

df['rolling_avg_5'] = df['rolling_avg_5'].fillna(df['career_avg'].mean())
print(df)

               batter  match_id        date                            venue  \
0      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
1      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
2      A Ashish Reddy    548346  2012-04-29                 Wankhede Stadium   
3      A Ashish Reddy    548352  2012-05-04  MA Chidambaram Stadium, Chepauk   
4      A Ashish Reddy    548352  2012-05-04  MA Chidambaram Stadium, Chepauk   
...               ...       ...         ...                              ...   
54066          Z Khan   1082595  2017-04-08            M.Chinnaswamy Stadium   
54067          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54068          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54069          Z Khan   1082635  2017-05-06                 Feroz Shah Kotla   
54070          Z Khan   1082646  2017-05-14                 Feroz Shah Kotla   

                bowler                 

In [18]:
#create target label
df['next_match_runs'] = df.groupby('batter')['batsman_runs'].shift(-1)

df= df.dropna()    
    

In [19]:
print(df)

               batter  match_id        date  \
1      A Ashish Reddy    548346  2012-04-29   
2      A Ashish Reddy    548346  2012-04-29   
4      A Ashish Reddy    548352  2012-05-04   
5      A Ashish Reddy    548359  2012-05-08   
6      A Ashish Reddy    548359  2012-05-08   
...               ...       ...         ...   
54065          Z Khan    980993  2016-05-15   
54066          Z Khan   1082595  2017-04-08   
54067          Z Khan   1082635  2017-05-06   
54068          Z Khan   1082635  2017-05-06   
54069          Z Khan   1082635  2017-05-06   

                                                   venue           bowler  \
1                                       Wankhede Stadium         MM Patel   
2                                       Wankhede Stadium      RJ Peterson   
4                        MA Chidambaram Stadium, Chepauk         DJ Bravo   
5              Rajiv Gandhi International Stadium, Uppal          P Awana   
6              Rajiv Gandhi International Stadium,

In [20]:
df['next_match_wicket'] = df.groupby('bowler')['is_wicket'].shift(-1)

df= df.dropna() 

In [21]:
print(df)

               batter  match_id        date  \
1      A Ashish Reddy    548346  2012-04-29   
2      A Ashish Reddy    548346  2012-04-29   
4      A Ashish Reddy    548352  2012-05-04   
5      A Ashish Reddy    548359  2012-05-08   
6      A Ashish Reddy    548359  2012-05-08   
...               ...       ...         ...   
54040          Z Khan    392192  2009-04-25   
54041          Z Khan    392203  2009-05-01   
54045          Z Khan    419165  2010-04-25   
54046          Z Khan    501208  2011-04-14   
54053          Z Khan    548324  2012-04-15   

                                           venue        bowler  \
1                               Wankhede Stadium      MM Patel   
2                               Wankhede Stadium   RJ Peterson   
4                MA Chidambaram Stadium, Chepauk      DJ Bravo   
5      Rajiv Gandhi International Stadium, Uppal       P Awana   
6      Rajiv Gandhi International Stadium, Uppal       P Kumar   
...                                    

In [22]:
#feature selection
features = df[['rolling_avg_5','venue_avg','pvt_avg','pvp_avg','career_avg']]
labels   = df[['next_match_runs', 'next_match_wicket']]


In [23]:
print("Features:")
print(features.head(5))

Features:
   rolling_avg_5  venue_avg  pvt_avg  pvp_avg  career_avg
1       6.052753   3.333333      3.0      0.0    3.000000
2       6.052753   3.333333      1.5      7.0    1.500000
4       2.600000   7.800000      2.0      8.0    3.000000
5       2.000000   4.227273      4.5      0.0    2.600000
6       3.200000   4.227273      0.0      6.0    2.166667


In [24]:
print("Labels:")
print(labels.head(5))

Labels:
   next_match_runs  next_match_wicket
1              7.0                0.0
2              2.0                1.0
4              0.0                0.0
5              6.0                0.0
6              2.0                0.0


In [25]:
train_df = df.sort_values('match_id').reset_index(drop=True)


In [26]:
train_df = train_df.sort_values('match_id')
split = int(len(df)*0.8)
X_train, X_test = features[:split], features[split:]
y_train, y_test = labels[:split], labels[split:]


In [27]:
#feature pipeline
pipeline = Pipeline([('scaler', StandardScaler())])

pipeline.fit(X_train)
joblib.dump(pipeline, r"C:\Users\ADMIN\Desktop\ipl_score\scripts\feature_pipeline.pkl")


['C:\\Users\\ADMIN\\Desktop\\ipl_score\\scripts\\feature_pipeline.pkl']

In [28]:
final_df = df[['batter','date','venue','bowling_team',
               'rolling_avg_5','venue_avg','pvt_avg','pvp_avg','career_avg']]

final_df.to_csv(
    r"C:\Users\ADMIN\Desktop\ipl_score\data\processed\dataset.csv",
    index=False
)


In [29]:
print(final_df.head())
print(final_df.shape)


           batter        date                                      venue  \
1  A Ashish Reddy  2012-04-29                           Wankhede Stadium   
2  A Ashish Reddy  2012-04-29                           Wankhede Stadium   
4  A Ashish Reddy  2012-05-04            MA Chidambaram Stadium, Chepauk   
5  A Ashish Reddy  2012-05-08  Rajiv Gandhi International Stadium, Uppal   
6  A Ashish Reddy  2012-05-08  Rajiv Gandhi International Stadium, Uppal   

          bowling_team  rolling_avg_5  venue_avg  pvt_avg  pvp_avg  career_avg  
1       Mumbai Indians       6.052753   3.333333      3.0      0.0    3.000000  
2       Mumbai Indians       6.052753   3.333333      1.5      7.0    1.500000  
4  Chennai Super Kings       2.600000   7.800000      2.0      8.0    3.000000  
5      Kings XI Punjab       2.000000   4.227273      4.5      0.0    2.600000  
6      Kings XI Punjab       3.200000   4.227273      0.0      6.0    2.166667  
(52866, 9)
